安装所需的软件包
安装必要的库。

2. 下载道琼斯 30 股票数据
使用 yfinance 下载道琼斯 30 公司的历史股票数据。

3. 从 CSV 文件加载数据
从 CSV 文件加载 Stock 数据。

4. 向数据添加技术指标
使用技术指标（RSI、MACD、CCI、ADX 等）增强数据。

5. 创建交易环境
设置用于模拟股票交易的环境。

6. 训练模型 — 集成 + PPO + A2C + DDPG + SAC + TD3
6.1 定义代理 定义 PPO、A2C、DDPG、SAC、TD3 和集成代理。

6.2 创建环境和训练智能体的功能 创建环境并训练 6 个智能体。

6.3 可视化投资组合变化的功能 可视化投资组合变化和净资产。

6.4 可视化代理性能的功能 可视化单个代理的性能。

6.5 测试和可视化 Ensemble Agent 的性能测试功能，并可视化 Ensemble Agent 的性能。

6.6 比较代理性能的功能 根据回报、标准差和夏普比率比较代理。

6.7 运行 执行训练和测试，可视化和比较结果。

7. 测试模型
在新数据上测试模型以评估性能。



In [1]:
import yfinance as yf 
import pandas as pd 
tickers = [
    'MMM','AXP','AAPL','BA','CAT','CVX','CSCO','KO','DIS','DOW',
    'GS','HD','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE',
    'PFE','PG','TRV','UNH','UTX','VZ','V','WBA','WMT','XOM'
]
def get_data(tickers):
    stock_data = {} 
    for ticker in tickers:
        df = yf.download(ticker,start="2009-01-01",end="2020-05-08") 
        stock_data[ticker] = df 
    return stock_data
stock_data = get_data(tickers) 
for ticker,df in stock_data.items():
    df.to_csv(f"D:/xuexi/[人工智能与算法交易]用于自动股票交易的深度强化学习/{ticker}.csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Could not get exchangeTimezoneName for ticker 'INTC' reason: 'chart'
[*********************100%********

In [2]:
import pandas as pd 
tickers = [
    'MMM','AXP','AAPL','BA','CAT','CVX','CSCO','KO','DIS','DOW',
    'GS','HD','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE',
    'PFE','PG','TRV','UNH','UTX','VZ','V','WBA','WMT','XOM'
]
benchmark = '^DJI' 
stock_data = {} 
for ticker in tickers:
    df = pd.read_csv(f'D:/xuexi/[人工智能与算法交易]用于自动股票交易的深度强化学习/{ticker}.csv',index_col='Date',parse_dates=True) 
    stock_data[ticker] = df 
training_data_time_range = ('2009-01-01','2015-12-31') 
validation_data_time_range = ('2016-01-01','2016-12-31') 
test_data_time_range = ('2017-01-01','2020-05-08') 

training_data = {} 
validation_data = {} 
test_data = {} 
for ticker,df in stock_data.items():
    training_data[ticker] = df.loc[training_data_time_range[0]:training_data_time_range[1]] 
    validation_data[ticker] = df.loc[validation_data_time_range[0]:validation_data_time_range[1]] 
    test_data[ticker] = df.loc[test_data_time_range[0]:test_data_time_range[1]] 
ticker = 'AAPL' 
print(f"Training data shape for {ticker}:{training_data[ticker].shape}") 
print(f'Validation data shape for {ticker}:{validation_data[ticker].shape}') 
print(f'Test data shape for {ticker}:{test_data[ticker].shape}') 

Training data shape for AAPL:(1762, 6)
Validation data shape for AAPL:(252, 6)
Test data shape for AAPL:(842, 6)


In [3]:
stock_data['AAPL'].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,3.067143,3.251429,3.041429,3.241071,2.737004,746015200
2009-01-05,3.327500,3.435000,3.311071,3.377857,2.852517,1181608400
2009-01-06,3.426786,3.470357,3.299643,3.322143,2.805468,1289310400
2009-01-07,3.278929,3.303571,3.223571,3.250357,2.744846,753048800
2009-01-08,3.229643,3.326786,3.215714,3.310714,2.795816,673500800


In [9]:
import numpy as np 
import pandas as pd 
def add_technical_indicators(df):
    delta = df['Close'].diff() 
    up = delta.where(delta > 0, 0) 
    down = -delta.where(delta < 0, 0) 
    rs = up.rolling(window=14).mean() / down.rolling(window=14).mean() 
    df['RSI'] = 100 - (100 / (1 + rs)) 
    df['EMA12'] = df['Close'].ewm(span=12,adjust=False).mean() 
    df['EMA26'] = df['Close'].ewm(span=26,adjust=False).mean() 
    df['MACD'] = df['EMA12'] - df['EMA26'] 
    df['Signal'] = df['MACD'].ewm(span=9,adjust=False).mean() 

    delta = df['Close'].diff() 
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean() 
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean() 
    rs = gain / loss 
    df['RSI'] = 100 - (100 / (1 + rs)) 

    tp = (df['High'] + df['Low'] + df['Close']) / 3 
    sma_tp = tp.rolling(window=20).mean() 
    mean_dev = tp.rolling(window=20).apply(lambda x: np.mean(np.abs(x- x.mean()))) 
    df['CCI'] = (tp - sma_tp) / (0.015 * mean_dev) 

    high_diff = df['High'].diff() 
    low_diff = df['Low'].diff() 
    df['+DM'] = np.where((high_diff > low_diff) & (high_diff > 0),high_diff, 0) 
    df['-DM'] = np.where((low_diff > high_diff) & (low_diff > 0),low_diff,0) 
    tr = pd.concat([df['High'] - df['Low'],np.abs(df['High'] - df['Close'].shift(1)),np.abs(df['Low'] - df['Close'].shift(1))],axis=1).max(axis=1)
    atr = tr.ewm(span=14,adjust=False).mean() 
    df['+DI'] = 100 * (df['+DM'].ewm(span=14,adjust=False).mean() / atr) 
    df['-DI'] = 100 * (df['-DM'].ewm(span=14,adjust=False).mean() / atr) 
    dx =100 * np.abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI']) 
    df['ADX'] = df.ewm(span=14,adjust=False).mean() 

    df.dropna(inplace=True) 
    df = df[['Open','High','Low','Close','Volume','MACD','Signal','RSI','CCI','ADX']] 
    return df 

In [10]:
for ticker,df in training_data.items():
    training_data[ticker] = add_technical_indicators(df) 
for ticker,df in validation_data.items():
    validation_data[ticker] = add_technical_indicators(df) 
for ticker,df in test_data.items():
    test_data[ticker] = add_technical_indicators(df) 
print("Shape of training data for AAPL:",training_data['AAPL'].shape) 
print('Shape of validation data for AAPL:',validation_data['AAPL'].shape) 
print('Shape of test data for AAPL:',test_data['AAPL'].shape)

C:\Users\石天辰\AppData\Local\Temp\ipykernel_13108\473600992.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = 100 - (100 / (1 + rs))
C:\Users\石天辰\AppData\Local\Temp\ipykernel_13108\473600992.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMA12'] = df['Close'].ewm(span=12,adjust=False).mean()
C:\Users\石天辰\AppData\Local\Temp\ipykernel_13108\473600992.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

ValueError: Cannot set a DataFrame with multiple columns to the single column ADX

In [6]:
df.reset_index(drop=True, inplace=True)


In [11]:
import gymnasium as gym 
from  gymnasium import spaces 
import numpy as np 
import pandas as pd 

class StockTradingEnv(gym.Env):
    metadata = {'render_modes':['human']} 
    def __init__(self,stock_data,transaction_cost_percent=0.005):
        super(StockTradingEnv,self).__init__() 

        self.stock_data = {ticker:df for ticker,df in stock_data.items() if not df.empty} 
        self.tickers = list(self.stock_data.keys()) 
        if not self.tickers:
            raise ValueError('All provided stock datat is empty') 

        sample_df = next(iter(self.stock_data.values())) 
        self.n_features = len(sample_df.columns) 
        self.action_space = spaces.Box(low=-1,high=1,shape=(len(self.tickers),),dtype=np.float32) 
        self.obs_shape = self.n_features * len(self.tickers) + 2 + len(self.tickers) + 2 
        self.observation_space = spaces.Box(low=-np.inf,high=np.inf,shape=(self.obs_shape,),dtype=np.float32) 
        self.initial_balance = 1000 
        self.balance = self.initial_balance 
        self.net_worth = self.initial_balance
        self.max_net_worth = self.initial_balance 
        self.shares_held = {ticker:0 for ticker in self.tickers} 
        self.total_shares_sold = {ticker: 0 for ticker in self.tickers} 
        self.total_sales_value = {ticker:0 for ticker in self.tickers} 

        self.current_step = 0 
        self.max_steps = max(0,min(len(df) for df in self.stock_data.values()) - 1) 
        self.transaction_cost_percent = transaction_cost_percent 

In [12]:
def reset(self,seed=None,options=None):
    super().reset(seed=seed) 
    self.balance = self.initial_balance 
    self.net_worth = self.initial_balance 
    self.max_net_worth = self.initial_balance 
    self.shares_held = {ticker:0 for ticker in self.tickers} 
    self.total_shares_sold = {ticker: 0 for ticker in self.tickers} 
    self.total_sales_value = {ticker:0 for ticker in self.tickers} 
    self.current_step = 0 
    return self._next_observation(),{} 

def _next_observation(self):
    frame = np.zeros(self.obs_shape) 
    idx = 0 
    for ticker in self.tickers:
        df = self.stock_data[ticker] 
        if self.current_step < len(df):
            frame[idx:idx+self.n_features] = df.iloc[self.current_step].values 
        elif len(df) > 0:
            frame[idx:idx+self.n_features] = df.iloc[-1].values 
        idx += self.n_features 
    frame[-4-len(self.tickers)] = self.balance 
    frame[-3-len(self.tickers):-3] = [self.shares_held[ticker] for ticker in self.tickers]
    frame[-3] = self.net_worth 
    frame[-2] = self.max_net_worth 
    frame[-1] = self.current_step 
    

In [13]:
def step(self,actions):
    self.current_step += 1 

    if self.current_step > self.max_steps:
        return self._next_observation(),0,True,False,{} 
    
    for i,ticker in enumerate(self.tickers):
        current_prices[ticker] = self.stock_data[ticker].iloc[self.current_step]['Close'] 
        action = actions[i] 

        if action > 0:
            shares_to_buy = int(self.balance * action / current_prices[ticker]) 
            cost = shares_to_buy * current_prices[ticker] 
            transaction_cost = cost * self.transaction_cost_percent 
            self.balance -= (cost + transaction_cost) 
            self.shares_held[ticker] += shares_to_buy 

        elif action < 0:
            shares_to_sell = int(self.shares_held[ticker] * abs(action)) 
            sale = shares_to_sell * current_prices[ticker] 
            transaction_cost = sale * self.transaction_cost_percent 
            self.balance += (sale - transaction_cost) 
            self.shares_held[ticker] -= shares_to_sell 
            self.total_shares_sold[ticker] += shares_to_sell 
            self.total_sales_value[ticker] += sale 
    self.net_worth = self.balance  + sum(self.shares_held[ticker] * current_prices[ticker] for ticker in self.tickers) 
    self.max_net_worth = max(self.net_worth,self.max_net_worth) 
    reward = self.net_worth - self.initial_balance 
    done = self.net_worth <= 0 or self.current_step >= self.max_steps 
    obs = self._next_observation() 
    return obs,reward,done,False,{}

In [14]:
def render(self,mode='human'):
    profit = self.net_worth - self.initial_balance 
    print(f'Step:{self.current_step}') 
    print(f'Balance:{self.balance:.2f}') 
    for ticker in self.tickers:
        print(f'{ticker} Shares held:{self.shares_held[ticker]}') 
    print(f'Net worth:{self.net_worth:.2f}') 
    print(f'Profit:{profit:.2f}') 
    
def close(self):
    pass 

In [15]:
from stable_baselines3 import PPO,A2C,DDPG,SAC,TD3 
from stable_baselines3.common.vec_env import  DummyVecEnv 
class EnsembleAgent:
    def __init__(self,ppo_model,a2c_model,ddpg_model,sac_model,td3_model):
        self.ppo_model = ppo_model 
        self.a2c_model = a2c_model 
        self.ddpg_model = ddpg_model 
        self.sac_model = sac_model 
        self.td3_model = td3_model 
    
    def predict(self,obs):
        ppo_action,_ = self.ppo_model.predict(obs) 
        a2c_action,_ = self.a2c_model.predict(obs) 
        ddpg_action,_ =  self.ddpg_model.predict(obs) 
        sac_action,_ = self.sac_model.predict(obs) 
        td3_action,_ = self.td3_model.predict(obs) 

        ensemble_action = np.mean([ppo_action,a2c_action,ddpg_action,sac_action,td3_action],axis=0) 
        return ensemble_action 
      

In [16]:
class PPOAgent:
    def __init__(self,env,total_timesteps):
        self.model = PPO("MlpPolicy",env,verbose=1) 
        self.model.learn(total_timesteps=total_timesteps) 
    
    def predict(self,obs):
        action,_ = self.model.predict(obs) 
        return action 

class A2CAgent:
    def __init__(self,env,total_timesteps):
        self.model = A2C("MlpPolicy",env,verbose=1) 
        self.model.learn(total_timesteps=total_timesteps) 

    def predict(self,obs):
        action,_ = self.model.predict(obs) 
        return action 

class DDPGAgent:
    def __init__(self,env,total_timesteps):
        self.model = DDPG("MlpPolicy",env,verbose=1) 
        self.model.learn(total_timesteps=total_timesteps) 

    def predict(self,obs):
        action,_ = self.model.predict(obs) 
        return action 

class SACAgent:
    def __init__(self,env,total_timesteps):
        self.model = SAC("MlpPolicy",env,verbose=1) 
        self.model.learn(total_timesteps=total_timesteps) 

    def predict(self,obs):
        action,_ = self.model.predict(obs) 
        return action 
class TD3Agent:
    def __init__(self,env,total_timesteps):
        self.model = TD3("MlpPolicy",env,verbose=1) 
        self.model.learn(total_timesteps=total_timesteps) 

    def predict(self,obs):
        action,_ = self.model.predict(obs) 
        return action 

In [17]:
import matplotlib.pyplot as plt 
def create_env_train_agents(data,total_timesteps):
    env = DummyVecEnv([lambda: StockTradingEnv(data)]) 
    ppo_agent = PPOAgent(env,total_timesteps) 
    a2c_agent = A2CAgent(env,total_timesteps) 
    ddpg_agent = DDPGAgent(env,total_timesteps) 
    sac_agent = SACAgent(env,total_timesteps) 
    td3_agent = TD3Agent(env,total_timesteps) 
    ensemble_agent = EnsembleAgent(ppo_agent.model,a2c_agent.model,ddpg_agent.model,sac_agent.model,td3_agent.model) 
    return env,ppo_agent,a2c_agent,ddpg_agent,sac_agent,td3_agent,ensemble_agent

In [18]:
def visualize_portfolio(steps,balances,net_worths,shares_held,tickers,show_balances=True,show_net_worth=True,show_shares_held=True):
    fig,axs = plt.subplots(3,figsize=(12,18)) 
    if show_balances: 
        axs[0].plot(steps,balances,label='Balance') 
        axs[0].set_title('Balance Over Time') 
        axs[0].set_xlabel('Steps') 
        axs[0].set_ylabel('Balance') 
        axs[0].legend() 
    if show_net_worth:
        axs[1].plot(steps,net_worths,label='Net Worth',color='orange') 
        axs[1].set_title('Net Worth Over Time') 
        axs[1].set_xlabel('Steps') 
        axs[1].set_ylabel('Net Worth') 
        axs[1].legend() 
    if show_shares_held:
        for ticker in tickers:
            axs[2].plot(steps,shares_held[tickers],label=f'Shares Held:{tickers}') 
        axs[2].set_title('Shares Held Over Time') 
        axs[2].set_xlabel('Steps') 
        axs[2].set_ylabel('shares Held') 
        axs[2].legend() 
    plt.tight_layout() 
    plt.show() 

def visualize_portfolio_worth(steps,net_worths):
    plt.figure(figsize=(12,6)) 
    plt.plot(steps,net_worths,label='Net Worth',color='orange') 
    plt.title('Net Worth over time') 
    plt.xlabel('Stpes') 
    plt.ylabel('Net Worth') 
    plt.legend() 
    plt.show() 

def visualize_multiple_portfolio_net_worth(steps,net_worths_list,labels):
    plt.figure(figsize=(12,6)) 
    for i,net_worths in enumerate(net_worths_list):
        plt.plot(steps,net_worths,label=labels[i]) 
    plt.title('Net Worth Over Time') 
    plt.xlabel('Steps') 
    plt.ylabel('Net Worth') 
    plt.legend() 
    plt.show() 


In [19]:
def test_agent(env,agent,stock_data,n_tests=1000,visualize=False):
    metrics = {
        'steps':[],
        'balances':[],
        'net_worths':[],
        'shares_held':{ticker:[] for ticker in stock_data.keys()} 
    }
    obs = env.reset() 
    for i in range(n_tests):
        metrics['steps'].append(i) 
        action = agent.predict(obs) 
        obs,rewards,dones,infos = env.step(action) 
        if visualize:
            env.render() 
        
        metrics['balances'].append(env.get_attr('balance')[0]) 
        metrics['net_worths'].append(env.get_attr('net_worth')[0]) 
        env_shares_held = env.get_attr('shares_held')[0] 

        for ticker in stock_data.keys():
            if ticker in env_shares_held:
                metrics['shares_held'][ticker].append(env_shares_held[ticker]) 
            else:
                metrics['shares_held'][ticker].append(0) 
        if dones:
            obs = env.reset() 
    return metrics 

In [20]:
def test_and_visualize_agents(env,agents,training_data,n_tests=100):
    metrics = {} 
    for agent_name,agent in agents.items():
        print(f"Testing {agent_name}...") 
        metrics[agent_name] = test_agent(env,agent,training_data,n_tests=n_tests,visualize=True) 
        print(f'Done testing{agent_name}') 
    print('-'*50) 
    print('All agents tested!') 
    print('-'*50) 

    net_worths = [metrics[agent_name]['net_worths'] for agent_name in agents.keys()] 
    steps = next(iter(metrics.values()))['steps'] 
    visualize_multiple_portfolio_net_worth(steps,net_worths,list(agents.keys()))